In [1]:
%pip install opencv-contrib-python
import cv2
import numpy as np

Note: you may need to restart the kernel to use updated packages.


In [58]:
#Read and import image
img = cv2.imread(r'C:\Users\MTBH.VN\OneDrive\desktop\Data learning\Python\Data 365\4. Python Programmer Bootcamp\Capstone project\19.1 capstone_coins.png', cv2.IMREAD_GRAYSCALE)
original = cv2.imread(r'C:\Users\MTBH.VN\OneDrive\desktop\Data learning\Python\Data 365\4. Python Programmer Bootcamp\Capstone project\19.1 capstone_coins.png', 1)

def get_radius(circle):
    radius = []
    for coord in circle[0,:]:
        radius.append(coord[2])
    return radius

def av_pix(img, circles, size):
    av_value = []
    for coord in circles[0,:]:
        col = np.mean(img[coord[1]-size:coord[1]+size,coord[0]-size:coord[0]+size])
        av_value.append(col)
    return av_value

#Convert to binary image and Blurring & Smoothing the picture to reduce noise (detail in the image)
img = cv2.GaussianBlur(img, (5,5) , 0)

#Detecting the edges of the image using Canny edge detection
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 0.9 ,120, param1=50, param2=27, minRadius=60, maxRadius=120)

#round the circle to integer values
circles = np.uint16(np.around(circles))
count = 0

print(circles)
#Draw the circle on the original image
#Because circles is a numpy array with 3 list 
'''
circle = [
           [[x, y, r], [x, y, r], ...]
         ]
'''
# So the first circles[0] is assess the first list '[[x, y, r], [x, y, r],...]', 
# and the second ':' is access all the elements in that list 
for i in circles[0,:]:
    print(i[0], i[1], i[2])
    #Draw the outter circle
    cv2.circle(original,(i[0], i[1]), i[2], (0, 255, 0), 2)
    #Draw the center of the circle
    cv2.circle(original,(i[0], i[1]), 2, (0, 0, 255), 3)
    #Add text to the image
    # cv2.putText(original, str(count), (i[0], i[1]), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2)
    count += 1

#Detect the polygon coin
'''Find a contour in the image, the area that contains the coin'''
imgcanny = cv2.Canny(img,20,20)

kernel = np.ones((5,5))

imgdill = cv2.dilate(imgcanny, kernel, iterations=1)
contours, hierachy = cv2.findContours(imgdill, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
contours_list = []
area_list = []
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 30000 and area < 50000:
        contours_list.append(cnt)
        area_list.append(area)

print(area_list)
print(contours_list)

polygon_Value = contours_list
for cnt in polygon_Value:
    perimeter = cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, 0.02 * perimeter, True)
    x,y,w,h = cv2.boundingRect(cnt)
print(x+y //2, y+h //2)

for cnt in polygon_Value:
    cv2.drawContours(original, cnt, -1, (0,255,0),2)

# Classify the coin base on the brightness and the radius of coin
# Value - Radius and brightness ???
radiii = get_radius(circles)
print(radiii)

brightness_value = av_pix(img, circles, 20)
print(brightness_value)

values = []
for a,b in zip(brightness_value, radiii):
    if a > 150 and b > 95:
        values.append(10)
    elif a > 150 and b <= 95:
        values.append(5)
    elif a <= 150 and b > 95:
        values.append(2)
    elif a <= 150 and b <= 95:
        values.append(1)

for cnt in contours_list:
    values.append(50)

count_2 = 0
for col in circles[0,:]:
    cv2.putText(original, str(values[count_2]) + 'P', (col[0], col[1]), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2)
    count_2 += 1

for cnt in polygon_Value:
    cv2.putText(original, str(values[count_2]) + 'P', (x+y //2, y+h //2), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2)

#Show the image with the circle drawn on it
cv2.putText(original, f'ESTIMATE TOTAL VALUE: {str(sum(values))} P', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255, 255, 255))

cv2.imshow('Detective coin', original)
cv2.imshow('Detective coin', original)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[[1060  200   96]
  [1108  500   99]
  [1312  398  103]
  [ 982  366   83]
  [ 764  500   80]
  [ 776  104   84]
  [ 940  580   81]
  [1272  220   67]]]
1060 200 96
1108 500 99
1312 398 103
982 366 83
764 500 80
776 104 84
940 580 81
1272 220 67
[39140.0]
[array([[[698, 172]],

       [[698, 173]],

       [[698, 174]],

       ...,

       [[701, 172]],

       [[700, 172]],

       [[699, 172]]], dtype=int32)]
665 286
[96, 99, 103, 83, 80, 84, 81, 67]
[179.844375, 85.68625, 97.176875, 131.384375, 73.96875, 101.205625, 75.08125, 181.275625]
